In [1]:
import requests
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from time import sleep
import random
import redis
import json

In [19]:
def get_classfy():
    classfy_list = []
    url = 'http://www.dianping.com/shenzhen/ch10'
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html')
    classfy = soup.find('div', id='classfy')
    for i in range(len(classfy.find_all('a'))):
        classfy_list.append(int(classfy.find_all('a')[i]['data-cat-id']))
    return classfy_list

In [20]:
get_classfy()

[112,
 117,
 103,
 111,
 132,
 113,
 110,
 116,
 219,
 251,
 508,
 114,
 102,
 1959,
 34236,
 2714,
 34014,
 33759,
 104,
 34055,
 25474,
 207,
 106,
 1338,
 101,
 1783,
 26481,
 34284,
 311,
 115,
 250,
 109,
 6743,
 34032,
 107,
 3243,
 34059,
 118]

In [27]:
# 获取各行政区单位id
def get_region_list(regionUrl):
    region_id_name = []
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(regionUrl, headers=headers)
    print(res.text)
    soup = BeautifulSoup(res.text, 'lxml')
    region = soup.find('div', class_='menu sub')
    for i in range(1, len(region.find_all('a', class_="item Fix"))):
        region_id_name.append(
            (int(region.find_all('a')[i]['data-itemid']), str(region.find_all('a')[i]['data-itemname'])))
    return region_id_name

In [28]:
# 将各个区域串起来
def get_all_area_list(regionurlList):
    all_area = []
    for regionname, regionurl in regionurlList:
        region_dict = {}
        region_id_name = get_region_list(regionurl)
        region_dict[regionname] = region_id_name
        all_area.append(region_dict)
    return all_area

In [29]:
def recostution_url(classfy_list, all_area):
    Reurl = []
    for classfy in classfy_list:
        for data in all_area:
            for region, regiondata in data.items():
                for area_id, area_name in regiondata:
                    Reurl.append((region, area_name, area_id,
                                  'http://m.dianping.com/shenzhen/ch10/' + 'g' + str(classfy) + 'r' + str(area_id)))
    return Reurl

In [24]:
def get_shopContent(Reurl):
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(Reurl, headers=headers)
    return res.text

In [31]:
pingshan = [('pingshan', 'http://www.dianping.com/shenzhen/ch10/r12035')]
classfy_list = get_classfy()

In [34]:
url = []
for number in classfy_list:
    url.append('http://www.dianping.com/shenzhen/ch10/g' + str(number) + 'r12035')
print(url)

['http://www.dianping.com/shenzhen/ch10/g112r12035', 'http://www.dianping.com/shenzhen/ch10/g117r12035', 'http://www.dianping.com/shenzhen/ch10/g103r12035', 'http://www.dianping.com/shenzhen/ch10/g111r12035', 'http://www.dianping.com/shenzhen/ch10/g132r12035', 'http://www.dianping.com/shenzhen/ch10/g113r12035', 'http://www.dianping.com/shenzhen/ch10/g110r12035', 'http://www.dianping.com/shenzhen/ch10/g116r12035', 'http://www.dianping.com/shenzhen/ch10/g219r12035', 'http://www.dianping.com/shenzhen/ch10/g251r12035', 'http://www.dianping.com/shenzhen/ch10/g508r12035', 'http://www.dianping.com/shenzhen/ch10/g114r12035', 'http://www.dianping.com/shenzhen/ch10/g102r12035', 'http://www.dianping.com/shenzhen/ch10/g1959r12035', 'http://www.dianping.com/shenzhen/ch10/g34236r12035', 'http://www.dianping.com/shenzhen/ch10/g2714r12035', 'http://www.dianping.com/shenzhen/ch10/g34014r12035', 'http://www.dianping.com/shenzhen/ch10/g33759r12035', 'http://www.dianping.com/shenzhen/ch10/g104r12035', 'ht

In [42]:
def get_shopContent(Reurl):
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(Reurl, headers=headers)
    #print(res.text)
    return res.text
def get_shopId(Reurl):
    resource = get_shopContent(Reurl)
    while '验证中心' in resource:
        print('出现验证码')
        sleep(random.randint(1, 3))
        resource = get_shopContent(Reurl)
        # 保存网页源码
    shopidList = []
    soup = BeautifulSoup(resource, 'lxml')
    p2 = re.compile(r'{.*}', re.S)
    string = soup.find_all('script')[2].string.strip()
    string = string.replace('true', 'True')
    string = string.replace('false', 'False')
    print()
    content = eval(re.findall(p2, string)[0])
    for adshop in content['mapiSearch']['data']['list']:
        shopidList.append(adshop['shopUuid'])
    return shopidList

In [43]:
shopidList = get_shopId('http://m.dianping.com/shenzhen/ch10/g112r12035')
print(shopidList)


['131322120', '20661171', '69486987', '130468605', '98400414', '97202024', '19805562', '2929828', '21561953', '132182063', '131747883', '113821775', '131341140', '69790217', '130216551', '8030397', '83590794', '125343751', '75065212', '127752744']
